In [1]:
import numpy as np
from collections import Counter, defaultdict

In [2]:
words = 'anpa ante awen esun insa jaki jelo kala kama kasi kili kule kute lape laso lawa lete lili lipu loje luka lupa mama mani meli mije moku moli musi mute nasa nena nimi noka olin open pali pana pini pipi poka poki pona sama seli selo seme sewi sike sina sona suli suno supa suwi taso tawa telo toki tomo unpa walo waso wawa weka wile'.split()
print(len(words))
chars = list(set([char for word in words for char in word]))
chars.sort()
print(len(chars))

66
14


In [3]:
###
### このソースコードには間違いがあるので
### wordle2.ipynb wordle_solution.ipynbのほうを参照してください
### 

def judge(ans, cand):
    show = [2, 2, 2, 2] # 0: green, 1: yellow, 2: no
    
    for i in range(4):
        if ans[i] == cand[i]:
            show[i] = 0
    
    for x in list(set(ans)):
        indices = [i for i in range(4) if cand[i] == x]
        if len(indices) > 0 and show[indices[0]] == 2:
            show[indices[0]] = 1
    
    j = ''
    chars = ['C', 'W', 'X']
    for n in show:
        j += chars[n]
    return j

print(judge('ante', 'wawa')) # 正解 ante に対して，候補 wawa を入力する

XWXX


In [4]:
def filter_words(words, j, cand):
    words = [x for x in words]
    
    # Wの定義を，厳密に「ここではなく他のところにある」にしてjを書き換え
    yellow_chars = ''
    for i in range(4):
        if j[i] == 'W' or j[i] == 'C':
            yellow_chars += cand[i]
    for i in range(4):
        if cand[i] in yellow_chars and j[i] == 'X':
            j = j[:i] + 'W' + j[i+1:]
    
    for i in range(4):
        if j[i] == 'C':
            words = [word for word in words if word[i] == cand[i]]
    
    for i in range(4):
        if j[i] == 'W':
            words = [word for word in words if cand[i] in word]
            words = [word for word in words if word[i] != cand[i]]
            
    for i in range(4):
        if j[i] == 'X':
            words = [word for word in words if cand[i] not in word]
            
    return words

print(filter_words(words, 'XWXX', 'wawa')) # 初手wawaに対する回答がXWXXだったとき，次に候補となる単語のリスト
print(filter_words(['kama', 'kasi'], 'CCXX', 'kama'))

['ante']
['kasi']


In [5]:
def ent(rest, cand):
    dct = defaultdict(int)
    
    for word in rest:
        dct[judge(word, cand)] += 1
        
    ent = 0
    denom = sum(dct.values())
    for i in dct.values():
        p = i / len(words)
        ent -= p * np.log2(p)
    return ent

print(ent(words, 'wawa')) # 全単語の中での wawa のエントロピー (情報量がすくない)
print(ent(words, 'laso')) # 全単語の中での laso のエントロピー (情報量がおおきい)

2.5385224930608845
4.441508514179351


In [7]:
rest = [x for x in words]

# 現状，最初の単語はkuleが最適と予測しているが，本当にそうかは謎
# 最初の2単語までのエントロピーをみたらkule,sonaが一番たかかったが，3単語以降まで考慮した場合どうなるのかはわからない
# 最初の入力単語として最適なのがなにかは誰か教えてください
word = input('input first word > ')
while True:
    j = input('input return > ')
    if j == '':
        break
    rest = filter_words(rest, j, word)
    cands = [(word, ent(rest, word)) for word in rest]
    cands.sort(key = lambda x: -x[-1])
    if len(cands) == 0:
        break
    elif len(cands) == 1:
        print('prediction: {} (entropy: {})'.format(*cands[0]))
        break
    else:
        print('prediciton: {} (entropy: {})'.format(*cands[0]))
        print('(others: ' + ', '.join(['{} ({})'.format(w, h) for w, h in cands[1:]]) + ')')
    word = cands[0][0]

input first word > sona
input return > XXXC
prediciton: kala (entropy: 0.7521918116979184)
(others: luka (0.7521918116979184), lawa (0.7333264708216073), weka (0.7333264708216073), kama (0.721888781394888), tawa (0.6915857510918577), wawa (0.6915857510918577), lupa (0.6483319181786252), mama (0.6309796904857972))
input return > XCWX
